In [3]:
!pip install ipykernel
!python -m ipykernel install --user --name=env --display-name "Python (env)"

Installed kernelspec env in C:\Users\ESHIKA\AppData\Roaming\jupyter\kernels\env


0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [4]:
!pip install Pillow pytesseract -qU langchain-groq langchain langchain_groq pdfplumber

In [5]:
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import re
import getpass
import os
from langchain_groq import ChatGroq
import pdfplumber
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

In [8]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["GROQ_API_KEY"] = ""
model = ChatGroq(model="llama3-70b-8192")

In [10]:
def extract_text_from_pdf(pdf_path): #for pdf
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ''
        for page in pdf.pages:
            all_text += page.extract_text() + '\n'
    return all_text


In [12]:
def extract_text_from_image(image_path): #for images
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text


In [31]:
def prompting(text): #prompting llama
  messages = [
      SystemMessage(content='''Extract all following values: Invoice no, Product,
      Quantity, Date, Unit rate, Amount Payable, Customer Details
      Customer Email, Customer phone number and Customer billing address from the data.

      Expected output : remove any dollar symbols {{'Invoice no':'1001329',
      'Product':'Office Chair', 'Quantity':'2', 'Date':'05/01/2022',
      'Unit rate':'1100.00', 'Amount Payable':'2300.00','Customer Details':'Test',
      'Customer Email':'santoshverma0988@gmail.com', 'Customer Phone number':'9999999999',
      'Customer billing Address':'Mumbai, India'}}
      Don't give any unnecessary comments.'''),
      HumanMessage(content=text),
  ]
  parser = StrOutputParser()
  result = model.invoke(messages)
  return parser.invoke(result)


In [16]:
def extraction(d,res): #using regex to extract values from generated string
  for k in d:
    st="'':\s*'([^']*)'"
    st1=st[:1]+k+st[1:]
    match = re.search(st1, res)
    if match:
      x = match.group(1)
      d[k]=x
    else:
      d[k]=None

In [47]:
import pandas as pd
def invoice_info(pdf_path):
  if pdf_path.endswith('.pdf'):
    invoice_text = extract_text_from_pdf(pdf_path)
  elif pdf_path.endswith('.jpg') or pdf_path.endswith('.jpeg') or pdf_path.endswith('.png'):
    invoice_text = extract_text_from_image(pdf_path)
  result=prompting(invoice_text)
  keyList = ['Invoice no', 'Product','Quantity','Date','Unit rate','Amount Payable','Customer Details','Customer Email','Customer Phone number','Customer billing Address']
  details = {}
  for i in keyList: details[i] = None
  extraction(details,result)
  return details

In [49]:
invoice_info('samp_inv.pdf')

{'Invoice no': 'inv41',
 'Product': 'WASTE AND SCRAP OF STAINLESS STEEL',
 'Quantity': '6,790 KGS',
 'Date': '18 Jul 2024',
 'Unit rate': '95.00',
 'Amount Payable': '768771.00',
 'Customer Details': 'TEST',
 'Customer Email': 'test@gmail.com',
 'Customer Phone number': '9108239284',
 'Customer billing Address': 'Hyderabad, TELANGANA, 500089'}

In [53]:
invoice_info('Sample_Invoice.pdf')

{'Invoice no': 'inv41',
 'Product': 'WASTE AND SCRAP OF STAINLESS STEEL',
 'Quantity': '6790.000 KGS',
 'Date': '18 Jul 2024',
 'Unit rate': '95.00',
 'Amount Payable': '7,68,771.00',
 'Customer Details': 'Test',
 'Customer Email': 'test@gmail.com',
 'Customer Phone number': '9108239284',
 'Customer billing Address': '36-TELANGANA, Hyderabad, TELANGANA, 500089'}

In [55]:
invoice_info('Sample_Invoice_image.png')

{'Invoice no': 'inva',
 'Product': 'WASTE AND SCRAP OF STAINLESS STEEL',
 'Quantity': '6790 KGS',
 'Date': '18. Jul 2024',
 'Unit rate': '95.00',
 'Amount Payable': '768771.00',
 'Customer Details': 'TEST',
 'Customer Email': 'test@gmail.com',
 'Customer Phone number': '9108239284',
 'Customer billing Address': 'Hyderabad, TELANGANA, 500089'}